In [ ]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path

REDIS_HOST='redis-17025.c258.us-east-1-4.ec2.cloud.redislabs.com'
REDIS_PORT=17025
REDIS_PASSWORD='Nsq7jK8HD3Idhz2pFBtVd3xBusUmmOqt'
# REDIS_HOST = 'redis-11938.c293.eu-central-1-1.ec2.cloud.redislabs.com'
# REDIS_PORT = 11938
REDIS_USERNAME = 'default'
# REDIS_PASSWORD = 'pZIaK9HYlVQnpVCLoyqjcAUJSYsyLfIi'

# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

# endpoint /status
class Status(object):
    exposed = True
    
    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

# endpoint /todos
class WriteResource(object):
    exposed = True

    def GET(self, *path, **query):
        keys = redis_client.keys('*:battery')
        
        keys = redis_client.keys('*battery')
        devices = []
        for k in keys:
            k = k.decode().split(":")[0]
            devices.append(k)
            # print("returning: ", item['devices'])
        
        devices = list(set(devices))

        response = {
            'mac_addresses' : devices
        }
        todo = json.dumps(response)

        # cherrypy.todo.status = "ok"       
        return todo


#endpoint /device
class Device(object):
    exposed = True
    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing id')
        mac_address = path[0]

        keys = redis_client.keys('*battery')
        devices = []
        for k in keys:
            k = k.decode().split(":")[0]
            devices.append(k)

        if mac_address not in devices:
            raise cherrypy.HTTPError(404 , 'Not Found: invalid MAC address')

        #Check if the start time is inserted
        from_ = query.get('from', -1)
        if int(from_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')
        
        #Check if the end time is inserted
        to_ = query.get('to', -1)
        if  int(to_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')

        battery_values = redis_client.ts().range(mac_address+':battery', from_time=from_, to_time=to_)
        power_values = redis_client.ts().range(mac_address+':power', from_time=from_, to_time=to_)
        
        
        response_dict = {
            'mac_address': mac_address,
            'timestamps': list(map(lambda x: x[0], battery_values)),
            'battery_levels': list(map(lambda x: x[1], battery_values)),
            'power_plugged':list(map(lambda x:x[1], power_values))
        }
        response = json.dumps(response_dict)


        cherrypy.response.status = "200 OK: Everything worked as expected"
        return response
    def DELETE(self, *path, **query):

        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')

        mac_address = path[0]

        #Try to remove the item
        battery_level = redis_client.delete(mac_address+':battery')
        power_level= redis_client.delete(mac_address+':power')

        #Check if the item has been removed
        if battery_level == 0 and power_level == 0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')

        return

if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(WriteResource(), '/write_resource', conf)
    cherrypy.tree.mount(Device(), '/device', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[27/Jan/2023:17:47:22] ENGINE Bus STARTING
[27/Jan/2023:17:47:22] ENGINE Started monitor thread 'Autoreloader'.
[27/Jan/2023:17:47:23] ENGINE Serving on http://0.0.0.0:8080
[27/Jan/2023:17:47:23] ENGINE Bus STARTED
Redis Connected: True


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1971cb16-e709-4b3f-8d6d-f6f260f3f6b7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>